# save the images as video
# Basler ace camera

In [6]:
# import packages and functions
from pypylon import pylon
from imageio import get_writer
import numpy as np
import os
import cv2
from datetime import datetime
import matplotlib.pyplot as plt
import sys
from camcommands import camOpen

connect camera with OpenCV

In [7]:
# list cameras connected with PC

# 1. get transport layer and all attached devices
tl_factory = pylon.TlFactory.GetInstance()
set_devices = tl_factory.EnumerateDevices() #enumerate devices connected

"""
# 1.1 software issue from pylon. this block may be deleted later
# remove duplicated camera list
set_devices = []
for device in devices:
    if device not in set_devices:
        set_devices.append(device)
    else:
        pass
set_devices = tuple(set_devices)
"""

# 2. check the devices information
if len(set_devices) == 0:
    raise pylon.RuntimeException("No camera connected")
else:
    print("total number of devices:", len(set_devices))
    for set_device in set_devices:
        print(set_device.GetFriendlyName()) # return readable name
        print(set_device.GetFullName()) # return unique code

total number of devices: 1
acA1300_SN01 (21566554)
Basler acA1300-30gc#00305317E75A#169.254.91.231:3956


In [8]:
# Create an array of instant cameras for the found devices and avoid exceeding a maximum number of devices.
cameras = pylon.InstantCameraArray(len(set_devices))
# Create and attach all Pylon Devices.
fps = 10 # frame per sec
ExposureTime = 5000 # in µs
Height, width = 962, 1286
for i, cam in enumerate(cameras):
    cam.Attach(tl_factory.CreateDevice(set_devices[i]))

    # Print the model name of the camera.
    print("Using device ", cam.GetDeviceInfo().GetFriendlyName())
    
    cam.Open() # open camera to get the value of the parameters and change with the new parameters
    cam.Height.SetValue(Height)
    cam.Width.SetValue(width)
    cam.ExposureTimeRaw.SetValue(ExposureTime)
    cam.AcquisitionFrameRateAbs.SetValue(fps)
    print(cam.Height.GetValue(), cam.Width.GetValue(), cam.ExposureTimeRaw.GetValue(), cam.AcquisitionFrameRateAbs.GetValue())
    cam.Close() # close camera after the change of parameter

Using device  acA1300_SN01 (21566554)
962 1286 5000 10.0


In [ ]:
# working with multiple cameras


In [10]:
for cam in cameras:
    with get_writer("/Users/dkim/Desktop/basler_camera/recording/apatuer_5000.avi") as writer:
        cam.StopGrabbing()
        cam.StartGrabbingMax(300)
        while cam.IsGrabbing():
            res = cam.RetrieveResult(300)
            writer.append_data(res.Array)
            #print(res.BlockID)
            res.Release()

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1286, 962) to (1296, 976) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
[swscaler @ 0x7fb008379000] Warning: data is not aligned! This can lead to a speed loss


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
